# Installation - Loading the Gymnasium custom environment: LifeSim

Since we're using a custom environment, we need to install its package into the pip environment, such that it will be usable from the gymnasium library. The following code is based on a Google Colab's run.

In [1]:
'''
# %%capture
#!rm -rf /content/gym_projects
!unzip - o / content/archive.zip
!pip uninstall - -no-input life-sim
!pip install - -no-input - e / content/gym_projects/
'''

'\n# %%capture\n#!rm -rf /content/gym_projects\n!unzip - o / content/archive.zip\n!pip uninstall - -no-input life-sim\n!pip install - -no-input - e / content/gym_projects/\n'

If the environment cannot be created with gymnasium.make(), it may be useful to restart the runtime (code for Colab).

In [2]:
'''
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()
'''


'\nimport os\n\ndef restart_runtime():\n  os.kill(os.getpid(), 9)\n\nrestart_runtime()\n'

# Creating the environment

In [3]:
import gymnasium as gym
import gymnasium.vector
import numpy as np
from collections import deque
import random
from life_sim.envs import LifeSim, LifeSteps
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from agent import Agent, PPO_Agent
import agent
import memory
import tensorflow as tf
import gc
import importlib
importlib.reload(gym)
importlib.reload(memory)
importlib.reload(agent)

import datetime
log_dir = "logs/scalars/final" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

2023-07-03 20:54:07.661349: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-03 20:54:08.221336: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu_devices[0], True)

2023-07-03 20:54:09.276925: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-03 20:54:09.298022: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-03 20:54:09.298178: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Set seeds for reproducibility.

In [5]:
seed = 14
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)


# seeds for testing
t_seeds = [int(x) for x in np.random.randint(1, 100000 + 1, size=1000)]

In [6]:
# this seeds can be used for testing the Agent in the notebook
# after every restart of the jupyter kernel (..happens) they
# will stay the same.
t_seeds[:10]

[79193, 9485, 22856, 44794, 89447, 78571, 55965, 95883, 99537, 39486]

Adding some useful settings and dictionaries to ease the code

In [7]:
# numpy settings for printing the states, if needed
np.set_printoptions(precision=8)

# some useful dictionaries used to let the code be more readable
# and less error-prone
num_actions = 3

actions = {
    "work": 0,
    "sport": 1,
    "sociality": 2
}

actions_array = {
    0: np.transpose([1, 0, 0]),
    1: np.transpose([0, 1, 0]),
    2: np.transpose([0, 0, 1]),
}

# Utils

Here we code some useful functions to visualize the results of the training and evaluate the model performance on the environment.

In [8]:
def print_evaluation(mcr, actions):

    print(f'mean cumulative reward: {mcr}')
    print(f'work was selected {actions[0]:.1%} of the time')
    print(f'sport was selected {actions[1]:.1%} of the time')
    print(f'sociality was selected {actions[2]:.1%} of the time')

In [9]:
def random_play(gamemode, difficulty, max_ts):
    env = gymnasium.make('life_steps/LifeSteps-v0', render_mode=None, max_timesteps=max_ts, difficulty=difficulty, gamemode=gamemode)

    cumulative_rewards = []

    n_episodes = 50

    for episode in tqdm(np.arange(1, n_episodes+1, 1), desc="Episodes", position=0):

        sum_rewards = 0
        actions = np.transpose([0, 0, 0])

        obs, info = env.reset()

        while True:    

            action = np.random.randint(3)
            new_obs, reward, terminated, truncated, info = env.step(action)

            sum_rewards += reward
    
            #env.render()
            if terminated or truncated:
               break

        cumulative_rewards.append(sum_rewards)
        
    env.close()
    
    print(f"Random reward in {gamemode} gamemode and difficulty {difficulty}: {np.mean(cumulative_rewards)}")

# Training

Before starting, let's check if the custom environment LifeSteps works well according to the Gymnasium API

In [10]:
from gymnasium.utils import env_checker
env = gymnasium.make('life_steps/LifeSteps-v0', max_timesteps=100, difficulty=0, gamemode='standard')
env_checker.check_env(env.unwrapped)
env.close()

| Life ->	M: 29,   H: 28,   S: 29  -   F: alone.., I just started playing!
| Life ->	M: 29,   H: 28,   S: 29  -   F: alone.., I just started playing!


In [11]:
env = gymnasium.make('life_steps/LifeSteps-v0', max_timesteps=100, difficulty=0, gamemode='monopoly')
env_checker.check_env(env.unwrapped)
env.close()

| Life ->	M: 32,   H: 29,   S: 32  -   F: alone.., I just started playing!
| Life ->	M: 32,   H: 29,   S: 32  -   F: alone.., I just started playing!


### Testing the random baselines

In [12]:
random_play('standard',40, 100)
random_play('monopoly',40, 100)

Episodes:   0%|          | 0/50 [00:00<?, ?it/s]

Random reward in standard gamemode and difficulty 40: -122.56


Episodes:   0%|          | 0/50 [00:00<?, ?it/s]

Random reward in monopoly gamemode and difficulty 40: -123.42


## Step 1 - Training in Standard Gamemode, difficulty 40

In [13]:
# max number of steps for each episode of the simulator.
# if reached, the environment is TRUNCATED by the TimeLimit wrapper
#max_steps = 300
num_envs = 32

env_max_timesteps = 100
difficulty = 40
gamemode = 'standard'

# create the vectorized environment
vec_env = gymnasium.vector.make(
    'life_steps/LifeSteps-v0', num_envs=num_envs, asynchronous=True, max_timesteps=env_max_timesteps, difficulty=difficulty, gamemode=gamemode)

#vec_env = gymnasium.vector.make(
#    'CartPole-v1', num_envs=num_envs, asynchronous=True)

In [14]:
input_shape = [vec_env.single_observation_space.shape[0]]
n_outputs = vec_env.single_action_space.n
print(f'input shape: {input_shape}')
print(f'num output: {n_outputs}')

input shape: [4]
num output: 3


In [15]:
num_steps = 1000000
# discount
gamma = 0.99
# lambda is the GAE parameter
lmbda = 0.95
# clipping parameter
epsilon = 0.2
# entropy parameter
c2 = 0.0
batch_timesteps = 128
minibatch_size = 32
epochs = 4

lr_actor = 2.5e-4
lr_critic = 2.5e-4

In [16]:
m = memory.Memory(num_envs = num_envs, obs_shape = (input_shape[0],), timesteps = batch_timesteps)

In [17]:
import agent
importlib.reload(agent)
ppo_agent = agent.PPO_Agent(input_shape, 
                            n_outputs, 
                            gamma, 
                            lmbda, 
                            epsilon, 
                            c2, 
                            lr_actor, 
                            lr_critic, 
                            log=True, 
                            log_dir=f'{log_dir}-sd{seed}-s1-{gamemode}-{difficulty}', 
                            env_max_timesteps=env_max_timesteps,
                            init_personal = True,
                            units=32)

2023-07-03 20:54:15.877503: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-03 20:54:15.877905: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-03 20:54:15.878059: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Train the model for the decided time.

In [18]:
#from agent import play_n_timesteps
#mean_losses_actor, mean_losses_critic = ppo_agent.play_n_timesteps(vec_env, m, num_steps, batch_timesteps, minibatch_size, epochs)

# with tensorboard logging
ppo_agent.play_n_timesteps(vec_env, m, num_steps, batch_timesteps, minibatch_size, epochs, difficulty = difficulty, gamemode = gamemode)

  0%|          | 0/7812 [00:00<?, ?it/s]

2023-07-03 20:54:19.746357: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f1b28016980 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-03 20:54:19.746393: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-07-03 20:54:19.749983: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-03 20:54:19.867506: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-03 20:54:19.915226: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-03 20:54:19.964904: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -123.500

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -3.400

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: 1.000

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: 1.000

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: 0.800

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: 1.000

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: 1.000

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Eval

### Evaluation - 1 
Evaluating the model for 50 episodes.

In [19]:
eval_episodes = 50

eval_seeds = t_seeds[:eval_episodes]
print(eval_seeds)

[79193, 9485, 22856, 44794, 89447, 78571, 55965, 95883, 99537, 39486, 71323, 62490, 66008, 48119, 10732, 33029, 40587, 79361, 61911, 9849, 51302, 6186, 33000, 8772, 34512, 95557, 70606, 11438, 68988, 16146, 54263, 22401, 93344, 82698, 49180, 41179, 42796, 32503, 75763, 12546, 3965, 86344, 47209, 96960, 24304, 43039, 54164, 11322, 22734, 80210]


In [20]:
mcr, mean_actions = ppo_agent.evaluate(eval_episodes, seeds=eval_seeds, render_mode='text', difficulty = difficulty, gamemode = gamemode)

print_evaluation(mcr, mean_actions)

STARTING EVALUATION

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Seed: 22734


| Life ->	M: 29,   H: 29,   S: 27  -   F: alone.., I just started playing!
| Life ->	M: 34,   H: 26,   S: 26  -   F: alone..		|  Last Action: work
| Life ->	M: 39,   H: 23,   S: 25  -   F: alone..		|  Last Action: work
| Life ->	M: 34,   H: 30,   S: 24  -   F: alone..		|  Last Action: sport
| Life ->	M: 39,   H: 27,   S: 23  -   F: alone..		|  Last Action: work
| Life ->	M: 34,   H: 34,   S: 22  -   F: alone..		|  Last Action: sport
| Life ->	M: 39,   H: 31,   S: 21  -   F: alone..		|  Last Action: work
| Life ->	M: 34,   H: 28,   S: 30  -   F: alone..		|  Last Action: social
| Life ->	M: 39,   H: 25,   S: 29  -   F: alone..		|  Last Action: work
| Life ->	M: 34,   H: 32,   S: 28  -   F: alone..		|  Last Action: sport
| Life ->	M: 39,   H: 29,   S: 27  -   F: alone..		|  Last Action: work

| Life ->	M: 43,   H: 32,   S: 25  -   F: alone..		|  Last Action: work
| Life ->	M: 38,   H: 29,   S: 34  -   F: alone..		|  Last Action: social
| Life ->	M: 43,   H: 26,   S: 33  -   F: alone..		|  Last Action: work
| Life ->	M: 38,   H: 33,   S: 32  -   F: alone..		|  Last Action: sport
| Life ->	M: 43,   H: 30,   S: 31  -   F: alone..		|  Last Action: work
| Life ->	M: 48,   H: 27,   S: 30  -   F: alone..		|  Last Action: work
| Life ->	M: 43,   H: 34,   S: 29  -   F: alone..		|  Last Action: sport
| Life ->	M: 48,   H: 31,   S: 28  -   F: alone..		|  Last Action: work
| Life ->	M: 43,   H: 28,   S: 37  -   F: alone..		|  Last Action: social
| Life ->	M: 48,   H: 25,   S: 36  -   F: alone..		|  Last Action: work
| Life ->	M: 43,   H: 32,   S: 35  -   F: alone..		|  Last Action: sport
| Life ->	M: 48,   H: 29,   S: 34  -   F: alone..		|  Last Action: work
| Life ->	M: 43,   H: 36,   S: 33  -   F: alone..		|  Last Action: sport
| Life ->	M: 48,   H: 33,   S: 32  -   F: alone..		|  La

| Life ->	M: 47,   H: 37,   S: 51  -   F: many!!!		|  Last Action: social
| Life ->	M: 52,   H: 34,   S: 50  -   F: many!!!		|  Last Action: work
| Life ->	M: 57,   H: 31,   S: 49  -   F: many!!!		|  Last Action: work
| Life ->	M: 52,   H: 38,   S: 48  -   F: many!!!		|  Last Action: sport
| Life ->	M: 57,   H: 35,   S: 47  -   F: many!!!		|  Last Action: work
| Life ->	M: 52,   H: 42,   S: 46  -   F: many!!!		|  Last Action: sport
| Life ->	M: 57,   H: 39,   S: 45  -   F: many!!!		|  Last Action: work
| Life ->	M: 52,   H: 46,   S: 44  -   F: many!!!		|  Last Action: sport
| Life ->	M: 57,   H: 43,   S: 43  -   F: many!!!		|  Last Action: work
| Life ->	M: 62,   H: 40,   S: 42  -   F: many!!!		|  Last Action: work
| Life ->	M: 57,   H: 47,   S: 41  -   F: many!!!		|  Last Action: sport
| Life ->	M: 62,   H: 44,   S: 40  -   F: many!!!		|  Last Action: work
| Life ->	M: 57,   H: 41,   S: 49  -   F: many!!!		|  Last Action: social
| Life ->	M: 52,   H: 48,   S: 48  -   F: many!!!		|  La

| Life ->	M: 63,   H: 50,   S: 49  -   F: many!!!		|  Last Action: sport
| Life ->	M: 68,   H: 47,   S: 48  -   F: many!!!		|  Last Action: work
| Life ->	M: 63,   H: 54,   S: 47  -   F: many!!!		|  Last Action: sport
| Life ->	M: 68,   H: 51,   S: 46  -   F: many!!!		|  Last Action: work
| Life ->	M: 73,   H: 48,   S: 45  -   F: many!!!		|  Last Action: work
| Life ->	M: 68,   H: 55,   S: 44  -   F: many!!!		|  Last Action: sport
| Life ->	M: 73,   H: 52,   S: 43  -   F: many!!!		|  Last Action: work
| Life ->	M: 68,   H: 59,   S: 42  -   F: many!!!		|  Last Action: sport
| Life ->	M: 63,   H: 66,   S: 41  -   F: many!!!		|  Last Action: sport
| Life ->	M: 68,   H: 63,   S: 40  -   F: many!!!		|  Last Action: work
| Life ->	M: 73,   H: 60,   S: 39  -   F: many!!!		|  Last Action: work
| Life ->	M: 78,   H: 57,   S: 38  -   F: many!!!		|  Last Action: work
| Life ->	M: 73,   H: 54,   S: 47  -   F: many!!!		|  Last Action: social
| Life ->	M: 68,   H: 51,   S: 56  -   F: many!!!		|  Las

| Life ->	M: 79,   H: 63,   S: 49  -   F: many!!!		|  Last Action: sport
| Life ->	M: 84,   H: 60,   S: 48  -   F: many!!!		|  Last Action: work
| Life ->	M: 79,   H: 57,   S: 57  -   F: many!!!		|  Last Action: social
| Life ->	M: 74,   H: 64,   S: 56  -   F: many!!!		|  Last Action: sport
| Life ->	M: 79,   H: 61,   S: 55  -   F: many!!!		|  Last Action: work
| Life ->	M: 84,   H: 58,   S: 54  -   F: many!!!		|  Last Action: work
| Life ->	M: 79,   H: 65,   S: 53  -   F: many!!!		|  Last Action: sport
| Life ->	M: 84,   H: 62,   S: 52  -   F: many!!!		|  Last Action: work

--! Game finished with reward 1 !--

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Seed: 66008


| Life ->	M: 34,   H: 25,   S: 31  -   F: alone.., I just started playing!
| Life ->	M: 39,   H: 22,   S: 30  -   F: alone..		|  Last Action: work
| Life ->	M: 34,   H: 29,   S: 29  -   F: alone..		|  

...

| Life ->	M: 66,   H: 42,   S: 56  -   F: many!!!		|  Last Action: social
| Life ->	M: 61,   H: 49,   S: 55  -   F: many!!!		|  Last Action: sport
| Life ->	M: 66,   H: 46,   S: 54  -   F: many!!!		|  Last Action: work
| Life ->	M: 61,   H: 53,   S: 53  -   F: many!!!		|  Last Action: sport
| Life ->	M: 66,   H: 50,   S: 52  -   F: many!!!		|  Last Action: work
| Life ->	M: 71,   H: 47,   S: 51  -   F: many!!!		|  Last Action: work
| Life ->	M: 66,   H: 54,   S: 50  -   F: many!!!		|  Last Action: sport
| Life ->	M: 71,   H: 51,   S: 49  -   F: many!!!		|  Last Action: work
| Life ->	M: 76,   H: 48,   S: 48  -   F: many!!!		|  Last Action: work
| Life ->	M: 71,   H: 55,   S: 47  -   F: many!!!		|  Last Action: sport
| Life ->	M: 76,   H: 52,   S: 46  -   F: many!!!		|  Last Action: work
| Life ->	M: 71,   H: 49,   S: 55  -   F: many!!!		|  Last Action: social
| Life ->	M: 66,   H: 56,   S: 54  -   F: many!!!		|  Last Action: sport
| Life ->	M: 71,   H: 53,   S: 53  -   F: many!!!		|  L

| Life ->	M: 41,   H: 37,   S: 26  -   F: alone..		|  Last Action: sport
| Life ->	M: 46,   H: 34,   S: 25  -   F: alone..		|  Last Action: work
| Life ->	M: 41,   H: 31,   S: 34  -   F: alone..		|  Last Action: social
| Life ->	M: 46,   H: 28,   S: 33  -   F: alone..		|  Last Action: work
| Life ->	M: 41,   H: 35,   S: 32  -   F: alone..		|  Last Action: sport
| Life ->	M: 46,   H: 32,   S: 31  -   F: alone..		|  Last Action: work
| Life ->	M: 41,   H: 29,   S: 40  -   F: alone..		|  Last Action: social
| Life ->	M: 36,   H: 36,   S: 39  -   F: alone..		|  Last Action: sport
| Life ->	M: 41,   H: 33,   S: 38  -   F: alone..		|  Last Action: work
| Life ->	M: 46,   H: 30,   S: 37  -   F: alone..		|  Last Action: work
| Life ->	M: 41,   H: 37,   S: 36  -   F: alone..		|  Last Action: sport
| Life ->	M: 46,   H: 34,   S: 35  -   F: alone..		|  Last Action: work
| Life ->	M: 51,   H: 31,   S: 34  -   F: alone..		|  Last Action: work
| Life ->	M: 46,   H: 28,   S: 43  -   F: many!!!		|  La

| Life ->	M: 50,   H: 40,   S: 50  -   F: many!!!		|  Last Action: sport
| Life ->	M: 55,   H: 37,   S: 49  -   F: many!!!		|  Last Action: work
| Life ->	M: 60,   H: 34,   S: 48  -   F: many!!!		|  Last Action: work
| Life ->	M: 55,   H: 41,   S: 47  -   F: many!!!		|  Last Action: sport
| Life ->	M: 60,   H: 38,   S: 46  -   F: many!!!		|  Last Action: work
| Life ->	M: 55,   H: 45,   S: 45  -   F: many!!!		|  Last Action: sport
| Life ->	M: 60,   H: 42,   S: 44  -   F: many!!!		|  Last Action: work
| Life ->	M: 65,   H: 39,   S: 43  -   F: many!!!		|  Last Action: work
| Life ->	M: 70,   H: 36,   S: 42  -   F: many!!!		|  Last Action: work
| Life ->	M: 65,   H: 43,   S: 41  -   F: many!!!		|  Last Action: sport
| Life ->	M: 60,   H: 50,   S: 40  -   F: many!!!		|  Last Action: sport
| Life ->	M: 65,   H: 47,   S: 39  -   F: many!!!		|  Last Action: work
| Life ->	M: 60,   H: 44,   S: 48  -   F: many!!!		|  Last Action: social
| Life ->	M: 65,   H: 41,   S: 47  -   F: many!!!		|  Las

| Life ->	M: 68,   H: 52,   S: 45  -   F: many!!!		|  Last Action: work
| Life ->	M: 63,   H: 49,   S: 54  -   F: many!!!		|  Last Action: social
| Life ->	M: 68,   H: 46,   S: 53  -   F: many!!!		|  Last Action: work
| Life ->	M: 73,   H: 43,   S: 52  -   F: many!!!		|  Last Action: work
| Life ->	M: 68,   H: 50,   S: 51  -   F: many!!!		|  Last Action: sport
| Life ->	M: 73,   H: 47,   S: 50  -   F: many!!!		|  Last Action: work
| Life ->	M: 68,   H: 54,   S: 49  -   F: many!!!		|  Last Action: sport
| Life ->	M: 73,   H: 51,   S: 48  -   F: many!!!		|  Last Action: work
| Life ->	M: 78,   H: 48,   S: 47  -   F: many!!!		|  Last Action: work
| Life ->	M: 73,   H: 55,   S: 46  -   F: many!!!		|  Last Action: sport
| Life ->	M: 78,   H: 52,   S: 45  -   F: many!!!		|  Last Action: work
| Life ->	M: 73,   H: 59,   S: 44  -   F: many!!!		|  Last Action: sport
| Life ->	M: 78,   H: 56,   S: 43  -   F: many!!!		|  Last Action: work
| Life ->	M: 73,   H: 53,   S: 52  -   F: many!!!		|  Last

| Life ->	M: 73,   H: 48,   S: 58  -   F: many!!!		|  Last Action: work
| Life ->	M: 68,   H: 55,   S: 57  -   F: many!!!		|  Last Action: sport
| Life ->	M: 73,   H: 52,   S: 56  -   F: many!!!		|  Last Action: work
| Life ->	M: 68,   H: 59,   S: 55  -   F: many!!!		|  Last Action: sport
| Life ->	M: 73,   H: 56,   S: 54  -   F: many!!!		|  Last Action: work
| Life ->	M: 78,   H: 53,   S: 53  -   F: many!!!		|  Last Action: work
| Life ->	M: 73,   H: 50,   S: 62  -   F: many!!!		|  Last Action: social
| Life ->	M: 68,   H: 57,   S: 61  -   F: many!!!		|  Last Action: sport
| Life ->	M: 73,   H: 54,   S: 60  -   F: many!!!		|  Last Action: work
| Life ->	M: 78,   H: 51,   S: 59  -   F: many!!!		|  Last Action: work

--! Game finished with reward 1 !--

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Seed: 42796


| Life ->	M: 28,   H: 26,   S: 26  -   F: alone.., I just

Two more simulations, one with difficulty decreased of 15 points, and the other with difficulty increased of 15, to understand how the model performs on different difficulties.

In [21]:
mcr, mean_actions = ppo_agent.evaluate(eval_episodes, seeds=eval_seeds, render_mode='text', difficulty = difficulty-15, gamemode = gamemode)

print_evaluation(mcr, mean_actions)

STARTING EVALUATION

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Seed: 96960


| Life ->	M: 28,   H: 33,   S: 25  -   F: alone.., I just started playing!
| Life ->	M: 33,   H: 30,   S: 24  -   F: alone..		|  Last Action: work
| Life ->	M: 38,   H: 27,   S: 23  -   F: alone..		|  Last Action: work
| Life ->	M: 33,   H: 24,   S: 32  -   F: alone..		|  Last Action: social
| Life ->	M: 38,   H: 21,   S: 31  -   F: alone..		|  Last Action: work
| Life ->	M: 33,   H: 28,   S: 30  -   F: alone..		|  Last Action: sport
| Life ->	M: 38,   H: 25,   S: 29  -   F: alone..		|  Last Action: work
| Life ->	M: 33,   H: 32,   S: 28  -   F: alone..		|  Last Action: sport
| Life ->	M: 38,   H: 29,   S: 27  -   F: alone..		|  Last Action: work
| Life ->	M: 43,   H: 26,   S: 26  -   F: alone..		|  Last Action: work
| Life ->	M: 38,   H: 33,   S: 25  -   F: alone..		|  Last Action: sport

| Life ->	M: 53,   H: 32,   S: 33  -   F: alone..		|  Last Action: work
| Life ->	M: 48,   H: 39,   S: 32  -   F: alone..		|  Last Action: sport
| Life ->	M: 53,   H: 36,   S: 31  -   F: alone..		|  Last Action: work
| Life ->	M: 48,   H: 43,   S: 30  -   F: alone..		|  Last Action: sport
| Life ->	M: 53,   H: 40,   S: 29  -   F: alone..		|  Last Action: work
| Life ->	M: 48,   H: 37,   S: 38  -   F: alone..		|  Last Action: social
| Life ->	M: 53,   H: 34,   S: 37  -   F: alone..		|  Last Action: work
| Life ->	M: 48,   H: 41,   S: 36  -   F: alone..		|  Last Action: sport
| Life ->	M: 53,   H: 38,   S: 35  -   F: alone..		|  Last Action: work
| Life ->	M: 48,   H: 35,   S: 44  -   F: many!!!		|  Last Action: social
| Life ->	M: 53,   H: 32,   S: 43  -   F: many!!!		|  Last Action: work
| Life ->	M: 48,   H: 39,   S: 42  -   F: many!!!		|  Last Action: sport
| Life ->	M: 53,   H: 36,   S: 41  -   F: many!!!		|  Last Action: work
| Life ->	M: 48,   H: 43,   S: 40  -   F: many!!!		|  La

| Life ->	M: 62,   H: 50,   S: 41  -   F: many!!!		|  Last Action: sport
| Life ->	M: 67,   H: 47,   S: 40  -   F: many!!!		|  Last Action: work
| Life ->	M: 72,   H: 44,   S: 39  -   F: many!!!		|  Last Action: work
| Life ->	M: 67,   H: 41,   S: 48  -   F: many!!!		|  Last Action: social
| Life ->	M: 62,   H: 48,   S: 47  -   F: many!!!		|  Last Action: sport
| Life ->	M: 67,   H: 45,   S: 46  -   F: many!!!		|  Last Action: work
| Life ->	M: 62,   H: 52,   S: 45  -   F: many!!!		|  Last Action: sport
| Life ->	M: 67,   H: 49,   S: 44  -   F: many!!!		|  Last Action: work
| Life ->	M: 72,   H: 46,   S: 43  -   F: many!!!		|  Last Action: work
| Life ->	M: 67,   H: 53,   S: 42  -   F: many!!!		|  Last Action: sport
| Life ->	M: 72,   H: 50,   S: 41  -   F: many!!!		|  Last Action: work
| Life ->	M: 67,   H: 57,   S: 40  -   F: many!!!		|  Last Action: sport
| Life ->	M: 62,   H: 54,   S: 49  -   F: many!!!		|  Last Action: social
| Life ->	M: 67,   H: 51,   S: 48  -   F: many!!!		|  L

| Life ->	M: 66,   H: 59,   S: 51  -   F: many!!!		|  Last Action: sport
| Life ->	M: 71,   H: 56,   S: 50  -   F: many!!!		|  Last Action: work
| Life ->	M: 76,   H: 53,   S: 49  -   F: many!!!		|  Last Action: work
| Life ->	M: 71,   H: 50,   S: 58  -   F: many!!!		|  Last Action: social
| Life ->	M: 76,   H: 47,   S: 57  -   F: many!!!		|  Last Action: work
| Life ->	M: 71,   H: 54,   S: 56  -   F: many!!!		|  Last Action: sport
| Life ->	M: 76,   H: 51,   S: 55  -   F: many!!!		|  Last Action: work
| Life ->	M: 71,   H: 58,   S: 54  -   F: many!!!		|  Last Action: sport
| Life ->	M: 76,   H: 55,   S: 53  -   F: many!!!		|  Last Action: work
| Life ->	M: 81,   H: 52,   S: 52  -   F: many!!!		|  Last Action: work
| Life ->	M: 76,   H: 59,   S: 51  -   F: many!!!		|  Last Action: sport
| Life ->	M: 81,   H: 56,   S: 50  -   F: many!!!		|  Last Action: work
| Life ->	M: 76,   H: 63,   S: 49  -   F: many!!!		|  Last Action: sport
| Life ->	M: 81,   H: 60,   S: 48  -   F: many!!!		|  Las

| Life ->	M: 49,   H: 34,   S: 20  -   F: alone..		|  Last Action: work
| Life ->	M: 44,   H: 31,   S: 29  -   F: alone..		|  Last Action: social
| Life ->	M: 39,   H: 28,   S: 38  -   F: alone..		|  Last Action: social
| Life ->	M: 34,   H: 35,   S: 37  -   F: alone..		|  Last Action: sport
| Life ->	M: 39,   H: 32,   S: 36  -   F: alone..		|  Last Action: work
| Life ->	M: 44,   H: 29,   S: 35  -   F: alone..		|  Last Action: work
| Life ->	M: 39,   H: 36,   S: 34  -   F: alone..		|  Last Action: sport
| Life ->	M: 44,   H: 33,   S: 33  -   F: alone..		|  Last Action: work
| Life ->	M: 49,   H: 30,   S: 32  -   F: alone..		|  Last Action: work
| Life ->	M: 44,   H: 37,   S: 31  -   F: alone..		|  Last Action: sport
| Life ->	M: 49,   H: 34,   S: 30  -   F: alone..		|  Last Action: work
| Life ->	M: 44,   H: 41,   S: 29  -   F: alone..		|  Last Action: sport
| Life ->	M: 49,   H: 38,   S: 28  -   F: alone..		|  Last Action: work
| Life ->	M: 44,   H: 35,   S: 37  -   F: alone..		|  La

...

| Life ->	M: 42,   H: 24,   S: 30  -   F: alone..		|  Last Action: work
| Life ->	M: 37,   H: 31,   S: 29  -   F: alone..		|  Last Action: sport
| Life ->	M: 42,   H: 28,   S: 28  -   F: alone..		|  Last Action: work
| Life ->	M: 37,   H: 35,   S: 27  -   F: alone..		|  Last Action: sport
| Life ->	M: 42,   H: 32,   S: 26  -   F: alone..		|  Last Action: work
| Life ->	M: 37,   H: 29,   S: 35  -   F: alone..		|  Last Action: social
| Life ->	M: 42,   H: 26,   S: 34  -   F: alone..		|  Last Action: work
| Life ->	M: 37,   H: 33,   S: 33  -   F: alone..		|  Last Action: sport
| Life ->	M: 42,   H: 30,   S: 32  -   F: alone..		|  Last Action: work
| Life ->	M: 37,   H: 37,   S: 31  -   F: alone..		|  Last Action: sport
| Life ->	M: 42,   H: 34,   S: 30  -   F: alone..		|  Last Action: work
| Life ->	M: 47,   H: 31,   S: 29  -   F: alone..		|  Last Action: work
| Life ->	M: 42,   H: 38,   S: 28  -   F: alone..		|  Last Action: sport
| Life ->	M: 47,   H: 35,   S: 27  -   F: alone..		|  Las

| Life ->	M: 50,   H: 36,   S: 39  -   F: many!!!		|  Last Action: work
| Life ->	M: 55,   H: 33,   S: 38  -   F: many!!!		|  Last Action: work
| Life ->	M: 50,   H: 40,   S: 37  -   F: many!!!		|  Last Action: sport
| Life ->	M: 55,   H: 37,   S: 36  -   F: many!!!		|  Last Action: work
| Life ->	M: 50,   H: 44,   S: 35  -   F: many!!!		|  Last Action: sport
| Life ->	M: 55,   H: 41,   S: 34  -   F: many!!!		|  Last Action: work
| Life ->	M: 50,   H: 38,   S: 43  -   F: many!!!		|  Last Action: social
| Life ->	M: 55,   H: 35,   S: 42  -   F: many!!!		|  Last Action: work
| Life ->	M: 50,   H: 42,   S: 41  -   F: many!!!		|  Last Action: sport
| Life ->	M: 55,   H: 39,   S: 40  -   F: many!!!		|  Last Action: work
| Life ->	M: 50,   H: 46,   S: 39  -   F: many!!!		|  Last Action: sport
| Life ->	M: 55,   H: 43,   S: 38  -   F: many!!!		|  Last Action: work
| Life ->	M: 60,   H: 40,   S: 37  -   F: many!!!		|  Last Action: work
| Life ->	M: 55,   H: 37,   S: 46  -   F: many!!!		|  Last

| Life ->	M: 54,   H: 45,   S: 37  -   F: many!!!		|  Last Action: sport
| Life ->	M: 59,   H: 42,   S: 36  -   F: many!!!		|  Last Action: work
| Life ->	M: 54,   H: 39,   S: 45  -   F: many!!!		|  Last Action: social
| Life ->	M: 49,   H: 46,   S: 44  -   F: many!!!		|  Last Action: sport
| Life ->	M: 54,   H: 43,   S: 43  -   F: many!!!		|  Last Action: work
| Life ->	M: 59,   H: 40,   S: 42  -   F: many!!!		|  Last Action: work
| Life ->	M: 54,   H: 47,   S: 41  -   F: many!!!		|  Last Action: sport
| Life ->	M: 59,   H: 44,   S: 40  -   F: many!!!		|  Last Action: work
| Life ->	M: 64,   H: 41,   S: 39  -   F: many!!!		|  Last Action: work
| Life ->	M: 59,   H: 48,   S: 38  -   F: many!!!		|  Last Action: sport
| Life ->	M: 64,   H: 45,   S: 37  -   F: many!!!		|  Last Action: work
| Life ->	M: 59,   H: 42,   S: 46  -   F: many!!!		|  Last Action: social
| Life ->	M: 54,   H: 49,   S: 45  -   F: many!!!		|  Last Action: sport
| Life ->	M: 59,   H: 46,   S: 44  -   F: many!!!		|  L

| Life ->	M: 63,   H: 53,   S: 49  -   F: many!!!		|  Last Action: sport
| Life ->	M: 68,   H: 50,   S: 48  -   F: many!!!		|  Last Action: work
| Life ->	M: 73,   H: 47,   S: 47  -   F: many!!!		|  Last Action: work
| Life ->	M: 68,   H: 54,   S: 46  -   F: many!!!		|  Last Action: sport
| Life ->	M: 73,   H: 51,   S: 45  -   F: many!!!		|  Last Action: work
| Life ->	M: 68,   H: 58,   S: 44  -   F: many!!!		|  Last Action: sport
| Life ->	M: 73,   H: 55,   S: 43  -   F: many!!!		|  Last Action: work
| Life ->	M: 68,   H: 52,   S: 52  -   F: many!!!		|  Last Action: social
| Life ->	M: 73,   H: 49,   S: 51  -   F: many!!!		|  Last Action: work
| Life ->	M: 68,   H: 56,   S: 50  -   F: many!!!		|  Last Action: sport
| Life ->	M: 73,   H: 53,   S: 49  -   F: many!!!		|  Last Action: work
| Life ->	M: 68,   H: 60,   S: 48  -   F: many!!!		|  Last Action: sport
| Life ->	M: 73,   H: 57,   S: 47  -   F: many!!!		|  Last Action: work
| Life ->	M: 78,   H: 54,   S: 46  -   F: many!!!		|  Las

| Life ->	M: 37,   H: 37,   S: 28  -   F: alone..		|  Last Action: sport
| Life ->	M: 42,   H: 34,   S: 27  -   F: alone..		|  Last Action: work
| Life ->	M: 47,   H: 31,   S: 26  -   F: alone..		|  Last Action: work
| Life ->	M: 42,   H: 28,   S: 35  -   F: alone..		|  Last Action: social
| Life ->	M: 37,   H: 35,   S: 34  -   F: alone..		|  Last Action: sport
| Life ->	M: 42,   H: 32,   S: 33  -   F: alone..		|  Last Action: work
| Life ->	M: 47,   H: 29,   S: 32  -   F: alone..		|  Last Action: work
| Life ->	M: 42,   H: 36,   S: 31  -   F: alone..		|  Last Action: sport
| Life ->	M: 47,   H: 33,   S: 30  -   F: alone..		|  Last Action: work
| Life ->	M: 42,   H: 30,   S: 39  -   F: alone..		|  Last Action: social
| Life ->	M: 37,   H: 37,   S: 38  -   F: alone..		|  Last Action: sport
| Life ->	M: 42,   H: 34,   S: 37  -   F: alone..		|  Last Action: work
| Life ->	M: 47,   H: 31,   S: 36  -   F: alone..		|  Last Action: work
| Life ->	M: 42,   H: 38,   S: 35  -   F: alone..		|  La

In [22]:
mcr, mean_actions = ppo_agent.evaluate(eval_episodes, seeds=eval_seeds, render_mode='text', difficulty = difficulty+15, gamemode = gamemode)

print_evaluation(mcr, mean_actions)

STARTING EVALUATION

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Seed: 34512


| Life ->	M: 29,   H: 31,   S: 34  -   F: alone.., I just started playing!
| Life ->	M: 34,   H: 28,   S: 33  -   F: alone..		|  Last Action: work
| Life ->	M: 39,   H: 25,   S: 32  -   F: alone..		|  Last Action: work
| Life ->	M: 34,   H: 32,   S: 31  -   F: alone..		|  Last Action: sport
| Life ->	M: 39,   H: 29,   S: 30  -   F: alone..		|  Last Action: work
| Life ->	M: 44,   H: 26,   S: 29  -   F: alone..		|  Last Action: work
| Life ->	M: 39,   H: 33,   S: 28  -   F: alone..		|  Last Action: sport
| Life ->	M: 44,   H: 30,   S: 27  -   F: alone..		|  Last Action: work
| Life ->	M: 39,   H: 27,   S: 36  -   F: alone..		|  Last Action: social
| Life ->	M: 44,   H: 24,   S: 35  -   F: alone..		|  Last Action: work
| Life ->	M: 39,   H: 31,   S: 34  -   F: alone..		|  Last Action: sport

| Life ->	M: 43,   H: 34,   S: 35  -   F: alone..		|  Last Action: social
| Life ->	M: 48,   H: 31,   S: 34  -   F: alone..		|  Last Action: work
| Life ->	M: 43,   H: 38,   S: 33  -   F: alone..		|  Last Action: sport
| Life ->	M: 38,   H: 45,   S: 32  -   F: alone..		|  Last Action: sport
| Life ->	M: 43,   H: 42,   S: 31  -   F: alone..		|  Last Action: work
| Life ->	M: 48,   H: 39,   S: 30  -   F: alone..		|  Last Action: work
| Life ->	M: 53,   H: 36,   S: 29  -   F: alone..		|  Last Action: work
| Life ->	M: 48,   H: 33,   S: 38  -   F: alone..		|  Last Action: social
| Life ->	M: 43,   H: 40,   S: 37  -   F: alone..		|  Last Action: sport
| Life ->	M: 48,   H: 37,   S: 36  -   F: alone..		|  Last Action: work
| Life ->	M: 53,   H: 34,   S: 35  -   F: alone..		|  Last Action: work
| Life ->	M: 48,   H: 41,   S: 34  -   F: alone..		|  Last Action: sport
| Life ->	M: 53,   H: 38,   S: 33  -   F: alone..		|  Last Action: work
| Life ->	M: 48,   H: 35,   S: 42  -   F: many!!!		|  La

| Life ->	M: 59,   H: 38,   S: 38  -   F: many!!!		|  Last Action: work
| Life ->	M: 54,   H: 45,   S: 37  -   F: many!!!		|  Last Action: sport
| Life ->	M: 59,   H: 42,   S: 36  -   F: many!!!		|  Last Action: work
| Life ->	M: 54,   H: 39,   S: 45  -   F: many!!!		|  Last Action: social
| Life ->	M: 49,   H: 46,   S: 44  -   F: many!!!		|  Last Action: sport
| Life ->	M: 54,   H: 43,   S: 43  -   F: many!!!		|  Last Action: work
| Life ->	M: 59,   H: 40,   S: 42  -   F: many!!!		|  Last Action: work
| Life ->	M: 54,   H: 47,   S: 41  -   F: many!!!		|  Last Action: sport
| Life ->	M: 59,   H: 44,   S: 40  -   F: many!!!		|  Last Action: work
| Life ->	M: 64,   H: 41,   S: 39  -   F: many!!!		|  Last Action: work
| Life ->	M: 59,   H: 48,   S: 38  -   F: many!!!		|  Last Action: sport
| Life ->	M: 64,   H: 45,   S: 37  -   F: many!!!		|  Last Action: work
| Life ->	M: 59,   H: 42,   S: 46  -   F: many!!!		|  Last Action: social
| Life ->	M: 54,   H: 49,   S: 45  -   F: many!!!		|  La

| Life ->	M: 72,   H: 46,   S: 51  -   F: many!!!		|  Last Action: work
| Life ->	M: 67,   H: 53,   S: 50  -   F: many!!!		|  Last Action: sport
| Life ->	M: 72,   H: 50,   S: 49  -   F: many!!!		|  Last Action: work
| Life ->	M: 67,   H: 57,   S: 48  -   F: many!!!		|  Last Action: sport
| Life ->	M: 72,   H: 54,   S: 47  -   F: many!!!		|  Last Action: work
| Life ->	M: 77,   H: 51,   S: 46  -   F: many!!!		|  Last Action: work
| Life ->	M: 72,   H: 48,   S: 55  -   F: many!!!		|  Last Action: social
| Life ->	M: 67,   H: 55,   S: 54  -   F: many!!!		|  Last Action: sport
| Life ->	M: 72,   H: 52,   S: 53  -   F: many!!!		|  Last Action: work
| Life ->	M: 77,   H: 49,   S: 52  -   F: many!!!		|  Last Action: work
| Life ->	M: 72,   H: 56,   S: 51  -   F: many!!!		|  Last Action: sport
| Life ->	M: 77,   H: 53,   S: 50  -   F: many!!!		|  Last Action: work
| Life ->	M: 82,   H: 50,   S: 49  -   F: many!!!		|  Last Action: work
| Life ->	M: 77,   H: 57,   S: 48  -   F: many!!!		|  Last

| Life ->	M: 74,   H: 58,   S: 50  -   F: many!!!		|  Last Action: work
| Life ->	M: 79,   H: 55,   S: 49  -   F: many!!!		|  Last Action: work
| Life ->	M: 74,   H: 62,   S: 48  -   F: many!!!		|  Last Action: sport
| Life ->	M: 79,   H: 59,   S: 47  -   F: many!!!		|  Last Action: work
| Life ->	M: 74,   H: 56,   S: 56  -   F: many!!!		|  Last Action: social
| Life ->	M: 79,   H: 53,   S: 55  -   F: many!!!		|  Last Action: work

--! Game finished with reward -2 !--

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Seed: 10732


| Life ->	M: 28,   H: 29,   S: 34  -   F: alone.., I just started playing!
| Life ->	M: 33,   H: 26,   S: 33  -   F: alone..		|  Last Action: work
| Life ->	M: 38,   H: 23,   S: 32  -   F: alone..		|  Last Action: work
| Life ->	M: 33,   H: 30,   S: 31  -   F: alone..		|  Last Action: sport
| Life ->	M: 38,   H: 27,   S: 30  -   F: alone..		|  

...

| Life ->	M: 58,   H: 48,   S: 42  -   F: many!!!		|  Last Action: sport
| Life ->	M: 63,   H: 45,   S: 41  -   F: many!!!		|  Last Action: work
| Life ->	M: 58,   H: 42,   S: 50  -   F: many!!!		|  Last Action: social
| Life ->	M: 63,   H: 39,   S: 49  -   F: many!!!		|  Last Action: work
| Life ->	M: 58,   H: 46,   S: 48  -   F: many!!!		|  Last Action: sport
| Life ->	M: 63,   H: 43,   S: 47  -   F: many!!!		|  Last Action: work
| Life ->	M: 58,   H: 50,   S: 46  -   F: many!!!		|  Last Action: sport
| Life ->	M: 63,   H: 47,   S: 45  -   F: many!!!		|  Last Action: work
| Life ->	M: 68,   H: 44,   S: 44  -   F: many!!!		|  Last Action: work
| Life ->	M: 63,   H: 51,   S: 43  -   F: many!!!		|  Last Action: sport
| Life ->	M: 68,   H: 48,   S: 42  -   F: many!!!		|  Last Action: work
| Life ->	M: 63,   H: 45,   S: 51  -   F: many!!!		|  Last Action: social
| Life ->	M: 58,   H: 52,   S: 50  -   F: many!!!		|  Last Action: sport
| Life ->	M: 63,   H: 49,   S: 49  -   F: many!!!		|  L

| Life ->	M: 68,   H: 58,   S: 50  -   F: many!!!		|  Last Action: sport
| Life ->	M: 73,   H: 55,   S: 49  -   F: many!!!		|  Last Action: work
| Life ->	M: 78,   H: 52,   S: 48  -   F: many!!!		|  Last Action: work
| Life ->	M: 73,   H: 59,   S: 47  -   F: many!!!		|  Last Action: sport
| Life ->	M: 78,   H: 56,   S: 46  -   F: many!!!		|  Last Action: work
| Life ->	M: 73,   H: 53,   S: 55  -   F: many!!!		|  Last Action: social
| Life ->	M: 68,   H: 60,   S: 54  -   F: many!!!		|  Last Action: sport
| Life ->	M: 73,   H: 57,   S: 53  -   F: many!!!		|  Last Action: work
| Life ->	M: 78,   H: 54,   S: 52  -   F: many!!!		|  Last Action: work
| Life ->	M: 73,   H: 51,   S: 61  -   F: many!!!		|  Last Action: social
| Life ->	M: 78,   H: 48,   S: 60  -   F: many!!!		|  Last Action: work
| Life ->	M: 73,   H: 55,   S: 59  -   F: many!!!		|  Last Action: sport
| Life ->	M: 78,   H: 52,   S: 58  -   F: many!!!		|  Last Action: work
| Life ->	M: 73,   H: 59,   S: 57  -   F: many!!!		|  La

| Life ->	M: 75,   H: 59,   S: 50  -   F: many!!!		|  Last Action: sport
| Life ->	M: 80,   H: 56,   S: 49  -   F: many!!!		|  Last Action: work
| Life ->	M: 75,   H: 53,   S: 58  -   F: many!!!		|  Last Action: social
| Life ->	M: 80,   H: 50,   S: 57  -   F: many!!!		|  Last Action: work
| Life ->	M: 75,   H: 57,   S: 56  -   F: many!!!		|  Last Action: sport
| Life ->	M: 80,   H: 54,   S: 55  -   F: many!!!		|  Last Action: work
| Life ->	M: 75,   H: 61,   S: 54  -   F: many!!!		|  Last Action: sport
| Life ->	M: 80,   H: 58,   S: 53  -   F: many!!!		|  Last Action: work
| Life ->	M: 75,   H: 65,   S: 52  -   F: many!!!		|  Last Action: sport
| Life ->	M: 80,   H: 62,   S: 51  -   F: many!!!		|  Last Action: work

--! Game finished with reward -4 !--

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Seed: 43039


| Life ->	M: 31,   H: 33,   S: 33  -   F: alone.., I ju

| Life ->	M: 41,   H: 29,   S: 23  -   F: alone..		|  Last Action: work
| Life ->	M: 36,   H: 26,   S: 32  -   F: alone..		|  Last Action: social
| Life ->	M: 41,   H: 23,   S: 31  -   F: alone..		|  Last Action: work
| Life ->	M: 36,   H: 30,   S: 30  -   F: alone..		|  Last Action: sport
| Life ->	M: 41,   H: 27,   S: 29  -   F: alone..		|  Last Action: work
| Life ->	M: 36,   H: 34,   S: 28  -   F: alone..		|  Last Action: sport
| Life ->	M: 41,   H: 31,   S: 27  -   F: alone..		|  Last Action: work
| Life ->	M: 46,   H: 28,   S: 26  -   F: alone..		|  Last Action: work
| Life ->	M: 41,   H: 25,   S: 35  -   F: alone..		|  Last Action: social
| Life ->	M: 36,   H: 32,   S: 34  -   F: alone..		|  Last Action: sport
| Life ->	M: 41,   H: 29,   S: 33  -   F: alone..		|  Last Action: work
| Life ->	M: 46,   H: 26,   S: 32  -   F: alone..		|  Last Action: work
| Life ->	M: 41,   H: 33,   S: 31  -   F: alone..		|  Last Action: sport
| Life ->	M: 46,   H: 30,   S: 30  -   F: alone..		|  La

| Life ->	M: 42,   H: 30,   S: 31  -   F: alone..		|  Last Action: work
| Life ->	M: 37,   H: 37,   S: 30  -   F: alone..		|  Last Action: sport
| Life ->	M: 42,   H: 34,   S: 29  -   F: alone..		|  Last Action: work
| Life ->	M: 47,   H: 31,   S: 28  -   F: alone..		|  Last Action: work
| Life ->	M: 42,   H: 28,   S: 37  -   F: alone..		|  Last Action: social
| Life ->	M: 37,   H: 35,   S: 36  -   F: alone..		|  Last Action: sport
| Life ->	M: 42,   H: 32,   S: 35  -   F: alone..		|  Last Action: work
| Life ->	M: 47,   H: 29,   S: 34  -   F: alone..		|  Last Action: work
| Life ->	M: 42,   H: 36,   S: 33  -   F: alone..		|  Last Action: sport
| Life ->	M: 47,   H: 33,   S: 32  -   F: alone..		|  Last Action: work
| Life ->	M: 42,   H: 40,   S: 31  -   F: alone..		|  Last Action: sport
| Life ->	M: 47,   H: 37,   S: 30  -   F: alone..		|  Last Action: work
| Life ->	M: 52,   H: 34,   S: 29  -   F: alone..		|  Last Action: work
| Life ->	M: 47,   H: 31,   S: 38  -   F: alone..		|  Last

Since we're happy with the results, let's save the actor and critic models, and close the vectorized environment.

In [23]:
vec_env.close()
ppo_agent.save_models(f"sd{seed}-{gamemode}-{difficulty}")

### Step 2 - Finetuning the Standard network for the Monopoly gamemode

Let's test the gamemode 'monopoly' with the network pretrained on the 'standard' gamemode.

In [24]:
#ppo_agent.load_models('standard-15')

In [25]:
mcr, mean_actions = ppo_agent.evaluate(eval_episodes, seeds=eval_seeds, render_mode='text', difficulty = difficulty, gamemode = 'monopoly')

print_evaluation(mcr, mean_actions)

STARTING EVALUATION

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Seed: 66008


| Life ->	M: 34,   H: 25,   S: 31  -   F: alone.., I just started playing!
| Life ->	M: 39,   H: 22,   S: 30  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 34,   H: 29,   S: 29  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 39,   H: 26,   S: 28  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 34,   H: 23,   S: 37  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 39,   H: 20,   S: 36  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 34,   H: 27,   S: 35  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 39,   H: 24,   S: 34  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 34,   H: 31,   S: 33  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 39,   H: 28,   S: 32  -   F: alone..		|  L

| Life ->	M: 32,   H: 24,   S: 26  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 37,   H: 21,   S: 25  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 28,   S: 24  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 37,   H: 25,   S: 23  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 22,   H: 32,   S: 22  -   F: alone..		|  Last Action: sport		| ...you lost some pennies.... -10 points loss
| Life ->	M: 27,   H: 29,   S: 21  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 26,   S: 20  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 27,   H: 23,   S: 29  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 32,   H: 20,   S: 28  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 37,   H: 17,   S: 27  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 24,   S: 26  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 37,   H: 21,   S: 25  -   F: alo

| Life ->	M: 37,   H: 23,   S: 22  -   F: alone..		|  Last Action: sport		| ...got hurt while cooking... -10 points loss
| Life ->	M: 32,   H: 30,   S: 21  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 37,   H: 27,   S: 20  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 24,   S: 29  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 37,   H: 21,   S: 28  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 28,   S: 27  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 37,   H: 25,   S: 26  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 32,   S: 25  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 37,   H: 29,   S: 24  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 26,   S: 23  -   F: alone..		|  Last Action: work		| ...you lost some pennies.... -10 points loss
| Life ->	M: 37,   H: 23,   S: 22  -   F: alone..		|  Last Action: work		| all ok 
| Life 

| Life ->	M: 36,   H: 16,   S: 25  -   F: alone..		|  Last Action: work		| ...got hurt while cooking... -10 points loss
| Life ->	M: 31,   H: 23,   S: 24  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 36,   H: 20,   S: 23  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 31,   H: 27,   S: 22  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 36,   H: 14,   S: 21  -   F: alone..		|  Last Action: work		| ...got hurt while cooking... -10 points loss
| Life ->	M: 31,   H: 21,   S: 20  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 26,   H: 18,   S: 29  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 31,   H: 15,   S: 28  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 26,   H: 22,   S: 27  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 31,   H: 9,   S: 26  -   F: alone..		|  Last Action: work		| ...got hurt while cooking... -10 points loss
| Life ->	M: 26,   H: 16,   S: 25  -   F: alone..		|  

| Life ->	M: 37,   H: 29,   S: 32  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 42,   H: 26,   S: 31  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 37,   H: 33,   S: 30  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 42,   H: 30,   S: 29  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 47,   H: 27,   S: 28  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 42,   H: 24,   S: 37  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 37,   H: 31,   S: 36  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 42,   H: 28,   S: 35  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 37,   H: 35,   S: 34  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 32,   H: 32,   S: 33  -   F: alone..		|  Last Action: work		| ...you lost some pennies.... -10 points loss
| Life ->	M: 37,   H: 29,   S: 32  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 42,   H: 26,   S: 31  -   F: a

...

| Life ->	M: 20,   H: 6,   S: 38  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 15,   H: 13,   S: 37  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 20,   H: 10,   S: 36  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 25,   H: 7,   S: 35  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 20,   H: 14,   S: 34  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 25,   H: 11,   S: 33  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 20,   H: 8,   S: 32  -   F: alone..		|  Last Action: sport		| ...got hurt while cooking... -10 points loss
| Life ->	M: 15,   H: 15,   S: 31  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 20,   H: 12,   S: 30  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 15,   H: 19,   S: 29  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 20,   H: 16,   S: 28  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 25,   H: 13,   S: 27  -   F: alone.

| Life ->	M: 39,   H: 37,   S: 29  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 44,   H: 34,   S: 28  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 49,   H: 31,   S: 27  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 44,   H: 38,   S: 26  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 49,   H: 35,   S: 25  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 44,   H: 32,   S: 34  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 49,   H: 29,   S: 33  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 44,   H: 36,   S: 32  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 49,   H: 33,   S: 31  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 44,   H: 40,   S: 30  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 49,   H: 37,   S: 29  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 44,   H: 34,   S: 38  -   F: many!!!		|  Last Action: social		| all 

| Life ->	M: 38,   H: 21,   S: 19  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 33,   H: 28,   S: 18  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 38,   H: 25,   S: 17  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 33,   H: 22,   S: 26  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 28,   H: 29,   S: 25  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 33,   H: 26,   S: 24  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 38,   H: 23,   S: 23  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 33,   H: 30,   S: 22  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 38,   H: 17,   S: 21  -   F: alone..		|  Last Action: work		| ...got hurt while cooking... -10 points loss
| Life ->	M: 33,   H: 24,   S: 20  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 28,   H: 21,   S: 29  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 33,   H: 18,   S: 28  -   F: 

| Life ->	M: 28,   H: 28,   S: 24  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 33,   H: 15,   S: 23  -   F: alone..		|  Last Action: work		| ...got hurt while cooking... -10 points loss
| Life ->	M: 28,   H: 22,   S: 22  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 23,   H: 29,   S: 21  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 28,   H: 26,   S: 20  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 33,   H: 23,   S: 19  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 28,   H: 30,   S: 18  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 33,   H: 27,   S: 17  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 28,   H: 24,   S: 26  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 33,   H: 21,   S: 25  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 28,   H: 28,   S: 24  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 33,   H: 15,   S: 23  -   F: a

| Life ->	M: 29,   H: 23,   S: 25  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 24,   H: 20,   S: 34  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 29,   H: 7,   S: 33  -   F: alone..		|  Last Action: work		| ...got hurt while cooking... -10 points loss
| Life ->	M: 24,   H: 14,   S: 32  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 19,   H: 21,   S: 31  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 24,   H: 18,   S: 30  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 29,   H: 15,   S: 29  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 14,   H: 22,   S: 28  -   F: alone..		|  Last Action: sport		| ...you lost some pennies.... -10 points loss
| Life ->	M: 19,   H: 19,   S: 27  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 24,   H: 16,   S: 26  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 19,   H: 13,   S: 25  -   F: alone..		|  Last Action: sport		| ...got hurt whil

In [26]:
# free some memory
del ppo_agent
gc.collect()

702

It's not working well, because the Monopoly gamemode is much more difficult and the strategy to beat the game is different, it needs the player to initially boost sociality so that the deficits on the life state will decrease!

We can try to fine-tune the network.

We'll create a new ppo_agent and reload the weights from the disk. In this way, if something goes wrong, we can try to change the parameters and do the fine tuning again.

In [27]:
num_steps = 300000
# discount
gamma = 0.99
# lambda is the GAE parameter
lmbda = 0.95
# clipping parameter
#epsilon = 0.3
epsilon = 0.2
# entropy parameter
c2 = 0.005
batch_timesteps = 128
minibatch_size = 32
epochs = 4

lr_actor = 1e-4
lr_critic = 1e-4

gamemode = 'monopoly'
difficulty = 40

ppo_agent_m = agent.PPO_Agent(input_shape, 
                            n_outputs, 
                            gamma, 
                            lmbda, 
                            epsilon, 
                            c2, 
                            lr_actor, 
                            lr_critic, 
                            log=True, 
                            log_dir=f'{log_dir}-sd{seed}-s2-{gamemode}-{difficulty}', 
                            env_max_timesteps=env_max_timesteps,
                            init_personal = True,
                            units=32)

ppo_agent_m.load_models(f'sd{seed}-standard-40')
m.reset()

vec_env_m = gymnasium.vector.make(
    'life_steps/LifeSteps-v0', num_envs=num_envs, asynchronous=True, max_timesteps=env_max_timesteps, difficulty = difficulty, gamemode=gamemode)

ppo_agent_m.play_n_timesteps(vec_env_m, m, num_steps, batch_timesteps, minibatch_size, epochs, difficulty = difficulty, gamemode = gamemode)

  0%|          | 0/2343 [00:00<?, ?it/s]

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -32.100

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -33.200

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -23.600

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -21.500

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -19.200

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -5.500

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -11.600

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

### Evaluation - 2
Evaluating the model for 50 episodes.

In [28]:
mcr, mean_actions = ppo_agent_m.evaluate(eval_episodes, seeds=eval_seeds, render_mode='text', difficulty = difficulty, gamemode = gamemode)

print_evaluation(mcr, mean_actions)

STARTING EVALUATION

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Seed: 51302


| Life ->	M: 31,   H: 27,   S: 29  -   F: alone.., I just started playing!
| Life ->	M: 26,   H: 24,   S: 38  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 31,   H: 21,   S: 37  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 26,   H: 28,   S: 36  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 31,   H: 25,   S: 35  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 26,   H: 32,   S: 34  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 31,   H: 29,   S: 33  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 16,   H: 36,   S: 32  -   F: alone..		|  Last Action: sport		| ...you lost some pennies.... -10 points loss
| Life ->	M: 21,   H: 33,   S: 31  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 26,  

| Life ->	M: 30,   H: 38,   S: 46  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 35,   H: 35,   S: 45  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 40,   H: 32,   S: 44  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 35,   H: 39,   S: 43  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 40,   H: 36,   S: 42  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 45,   H: 33,   S: 41  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 40,   H: 40,   S: 40  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 35,   H: 37,   S: 49  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 40,   H: 32,   S: 48  -   F: many!!!		|  Last Action: work		| ...got hurt while cooking...but Dave helped, -2 points loss
| Life ->	M: 35,   H: 39,   S: 47  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 40,   H: 36,   S: 46  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 35,   H: 43,   S

| Life ->	M: 45,   H: 46,   S: 49  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 50,   H: 43,   S: 48  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 45,   H: 40,   S: 57  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 40,   H: 47,   S: 56  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 45,   H: 44,   S: 55  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 50,   H: 41,   S: 54  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 45,   H: 48,   S: 53  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 40,   H: 45,   S: 62  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 42,   H: 42,   S: 61  -   F: many!!!		|  Last Action: work		| ...you lost some pennies....but Dave helped, -3 points loss
| Life ->	M: 47,   H: 39,   S: 60  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 42,   H: 46,   S: 59  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 47,   H: 43,  

| Life ->	M: 29,   H: 30,   S: 31  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 34,   H: 27,   S: 30  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 29,   H: 24,   S: 39  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 24,   H: 31,   S: 38  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 19,   H: 28,   S: 47  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 24,   H: 25,   S: 46  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 29,   H: 20,   S: 45  -   F: many!!!		|  Last Action: work		| ...got hurt while cooking...but Piz helped, -2 points loss
| Life ->	M: 24,   H: 27,   S: 44  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 29,   H: 24,   S: 43  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 24,   H: 31,   S: 42  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 29,   H: 28,   S: 41  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 34,   H: 25,   

| Life ->	M: 34,   H: 44,   S: 40  -   F: many!!!		|  Last Action: work		| ...you lost some pennies....but Dave helped, -2 points loss
| Life ->	M: 39,   H: 41,   S: 39  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 44,   H: 38,   S: 38  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 39,   H: 35,   S: 47  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 34,   H: 38,   S: 46  -   F: many!!!		|  Last Action: sport		| ...got hurt while cooking...but Dave helped, -4 points loss
| Life ->	M: 39,   H: 35,   S: 45  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 44,   H: 32,   S: 44  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 39,   H: 39,   S: 43  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 44,   H: 34,   S: 42  -   F: many!!!		|  Last Action: work		| ...got hurt while cooking...but Leandro helped, -2 points loss
| Life ->	M: 39,   H: 41,   S: 41  -   F: many!!!		|  Last Action: sport		| all ok 
| Life

...

| Life ->	M: 40,   H: 37,   S: 51  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 45,   H: 31,   S: 50  -   F: many!!!		|  Last Action: work		| ...got hurt while cooking...but Dave helped, -3 points loss
| Life ->	M: 40,   H: 38,   S: 49  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 35,   H: 35,   S: 58  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 40,   H: 32,   S: 57  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 35,   H: 39,   S: 56  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 40,   H: 36,   S: 55  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 35,   H: 43,   S: 54  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 40,   H: 40,   S: 53  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 45,   H: 37,   S: 52  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 50,   H: 34,   S: 51  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 45,   H: 28,   S:

| Life ->	M: 39,   H: 36,   S: 42  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 44,   H: 33,   S: 41  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 39,   H: 40,   S: 40  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 30,   H: 37,   S: 49  -   F: many!!!		|  Last Action: social		| ...you lost some pennies....but Dudu helped, -4 points loss
| Life ->	M: 25,   H: 44,   S: 48  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 30,   H: 41,   S: 47  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 35,   H: 38,   S: 46  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 40,   H: 35,   S: 45  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 35,   H: 42,   S: 44  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 30,   H: 49,   S: 43  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 35,   H: 46,   S: 42  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 40,   H: 43,   S

| Life ->	M: 26,   H: 45,   S: 49  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 21,   H: 42,   S: 58  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 26,   H: 39,   S: 57  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 31,   H: 36,   S: 56  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 36,   H: 33,   S: 55  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 41,   H: 30,   S: 54  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 36,   H: 37,   S: 53  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 41,   H: 34,   S: 52  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 36,   H: 41,   S: 51  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 41,   H: 36,   S: 50  -   F: many!!!		|  Last Action: work		| ...got hurt while cooking...but Piz helped, -2 points loss
| Life ->	M: 36,   H: 43,   S: 49  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 41,   H: 40,   S

| Life ->	M: 15,   H: 21,   S: 38  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 20,   H: 18,   S: 37  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 15,   H: 25,   S: 36  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 20,   H: 22,   S: 35  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 25,   H: 19,   S: 34  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 20,   H: 26,   S: 33  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 15,   H: 23,   S: 32  -   F: alone..		|  Last Action: work		| ...you lost some pennies.... -10 points loss
| Life ->	M: 20,   H: 20,   S: 31  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 15,   H: 17,   S: 40  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 20,   H: 14,   S: 39  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 15,   H: 21,   S: 38  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 10,   H: 18,   S: 47  -   F: man

| Life ->	M: 51,   H: 40,   S: 48  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 46,   H: 47,   S: 47  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 51,   H: 44,   S: 46  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 56,   H: 41,   S: 45  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 51,   H: 48,   S: 44  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 46,   H: 45,   S: 53  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 51,   H: 42,   S: 52  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 56,   H: 39,   S: 51  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 51,   H: 46,   S: 50  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 56,   H: 43,   S: 49  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 51,   H: 50,   S: 48  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 46,   H: 47,   S: 57  -   F: many!!!		|  Last Action: social		| all 

The agent behaviour is not optimal but not even bad.

In [29]:
ppo_agent_m.save_models(f"sd{seed}-standardto{gamemode}-{difficulty}")

In [30]:
del ppo_agent_m
gc.collect()

584

### Step 3 - Restart from a brand new PPO architecture

Note: Tensorflow 'deterministic' behaviour (we're setting the seed at the start of the notebook) results in a big memory leak. For this reason, the notebook was restarted here to empty the memory and keep the training going.

In [31]:
num_steps = 1000000
# discount
gamma = 0.99
# lambda is the GAE parameter
lmbda = 0.95
# clipping parameter
epsilon = 0.2
# entropy parameter
c2 = 0.005
batch_timesteps = 128
minibatch_size = 32
epochs = 4

lr_actor = 2.5e-4
lr_critic = 2.5e-4

gamemode = 'monopoly'
difficulty = 40

vec_env_m = gymnasium.vector.make(
    'life_steps/LifeSteps-v0', num_envs=num_envs, asynchronous=True, max_timesteps = env_max_timesteps, difficulty = difficulty, gamemode = gamemode)

#m = memory.Memory(num_envs = num_envs, obs_shape = (input_shape[0],), timesteps = batch_timesteps)
m.reset()

ppo_agent_m = agent.PPO_Agent(input_shape, 
                                n_outputs, 
                                gamma, 
                                lmbda, 
                                epsilon, 
                                c2, 
                                lr_actor, 
                                lr_critic, 
                                log=True, 
                                log_dir=f'{log_dir}-sd{seed}-s3-{gamemode}-{difficulty}', 
                                env_max_timesteps=env_max_timesteps,
                                init_personal = True,
                                units=32)

ppo_agent_m.play_n_timesteps(vec_env_m, m, num_steps, batch_timesteps, minibatch_size, epochs, difficulty = difficulty, gamemode = gamemode)

  0%|          | 0/7812 [00:00<?, ?it/s]

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -123.800

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -32.800

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -26.100

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -22.600

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -0.400

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -7.600

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -7.000

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


### Evaluation - 3

In [32]:
mcr, mean_actions = ppo_agent_m.evaluate(eval_episodes,  seeds=eval_seeds, render_mode='text', difficulty=difficulty, gamemode=gamemode)

print_evaluation(mcr, mean_actions)

STARTING EVALUATION

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Seed: 41179


| Life ->	M: 31,   H: 33,   S: 28  -   F: alone.., I just started playing!
| Life ->	M: 26,   H: 30,   S: 37  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 31,   H: 27,   S: 36  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 26,   H: 34,   S: 35  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 21,   H: 31,   S: 34  -   F: alone..		|  Last Action: work		| ...you lost some pennies.... -10 points loss
| Life ->	M: 26,   H: 28,   S: 33  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 31,   H: 15,   S: 32  -   F: alone..		|  Last Action: work		| ...got hurt while cooking... -10 points loss
| Life ->	M: 26,   H: 12,   S: 41  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 21,   H: 19,   S: 40  -   F: many!!!		|  Last Actio

| Life ->	M: 35,   H: 29,   S: 46  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 30,   H: 36,   S: 45  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 35,   H: 33,   S: 44  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 30,   H: 40,   S: 43  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 35,   H: 33,   S: 42  -   F: many!!!		|  Last Action: work		| ...got hurt while cooking...but Dudu helped, -4 points loss
| Life ->	M: 30,   H: 40,   S: 41  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 35,   H: 37,   S: 40  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 40,   H: 34,   S: 39  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 35,   H: 31,   S: 48  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 30,   H: 38,   S: 47  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 25,   H: 45,   S: 46  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 30,   H: 42, 

| Life ->	M: 41,   H: 41,   S: 49  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 36,   H: 48,   S: 48  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 31,   H: 42,   S: 57  -   F: many!!!		|  Last Action: social		| ...got hurt while cooking...but Piz helped, -3 points loss
| Life ->	M: 36,   H: 39,   S: 56  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 41,   H: 36,   S: 55  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 36,   H: 43,   S: 54  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 41,   H: 40,   S: 53  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 46,   H: 37,   S: 52  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 41,   H: 44,   S: 51  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 46,   H: 41,   S: 50  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 41,   H: 48,   S: 49  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 42,   H: 45,   S:

| Life ->	M: 40,   H: 49,   S: 49  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 45,   H: 46,   S: 48  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 40,   H: 43,   S: 57  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 45,   H: 40,   S: 56  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 40,   H: 47,   S: 55  -   F: many!!!		|  Last Action: sport		| all ok 

--! Game finished with reward 1 !--

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Seed: 49180


| Life ->	M: 29,   H: 32,   S: 25  -   F: alone.., I just started playing!
| Life ->	M: 24,   H: 29,   S: 34  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 29,   H: 26,   S: 33  -   F: alone..		|  Last Action: work		| all ok 
| Life ->	M: 24,   H: 33,   S: 32  -   F: alone..		|  Last Action: sport		| all ok 
| Life ->	M: 29,   H: 30,   S: 31  -

| Life ->	M: 34,   H: 34,   S: 50  -   F: many!!!		|  Last Action: social		| ...you lost some pennies....but Dave helped, -2 points loss
| Life ->	M: 39,   H: 31,   S: 49  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 34,   H: 38,   S: 48  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 39,   H: 35,   S: 47  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 34,   H: 38,   S: 46  -   F: many!!!		|  Last Action: sport		| ...got hurt while cooking...but Dave helped, -4 points loss
| Life ->	M: 39,   H: 35,   S: 45  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 44,   H: 32,   S: 44  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 39,   H: 29,   S: 53  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 34,   H: 34,   S: 52  -   F: many!!!		|  Last Action: sport		| ...got hurt while cooking...but Leandro helped, -2 points loss
| Life ->	M: 39,   H: 31,   S: 51  -   F: many!!!		|  Last Action: work		| all ok 
| Li

...

| Life ->	M: 31,   H: 40,   S: 37  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 36,   H: 37,   S: 36  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 28,   H: 44,   S: 35  -   F: many!!!		|  Last Action: sport		| ...you lost some pennies....but Dudu helped, -3 points loss
| Life ->	M: 33,   H: 41,   S: 34  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 38,   H: 38,   S: 33  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 33,   H: 35,   S: 42  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 38,   H: 32,   S: 41  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 33,   H: 29,   S: 50  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 38,   H: 26,   S: 49  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 33,   H: 33,   S: 48  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 34,   H: 30,   S: 47  -   F: many!!!		|  Last Action: work		| ...you lost some pennies....but Lean

| Life ->	M: 38,   H: 50,   S: 46  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 43,   H: 47,   S: 45  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 48,   H: 44,   S: 44  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 43,   H: 51,   S: 43  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 38,   H: 48,   S: 52  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 43,   H: 45,   S: 51  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 48,   H: 42,   S: 50  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 43,   H: 49,   S: 49  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 48,   H: 46,   S: 48  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 43,   H: 53,   S: 47  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 48,   H: 49,   S: 46  -   F: many!!!		|  Last Action: work		| ...got hurt while cooking...but Piz helped, -1 points loss
| Life ->	M: 43,   H: 46,   S:

| Life ->	M: 31,   H: 31,   S: 39  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 36,   H: 28,   S: 38  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 27,   H: 35,   S: 37  -   F: many!!!		|  Last Action: sport		| ...you lost some pennies....but Dave helped, -4 points loss
| Life ->	M: 32,   H: 32,   S: 36  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 27,   H: 39,   S: 35  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 32,   H: 36,   S: 34  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 37,   H: 33,   S: 33  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 30,   S: 42  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 37,   H: 27,   S: 41  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 24,   S: 50  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 37,   H: 21,   S: 49  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 28,   S

| Life ->	M: 34,   H: 39,   S: 49  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 39,   H: 36,   S: 48  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 34,   H: 43,   S: 47  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 39,   H: 40,   S: 46  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 44,   H: 37,   S: 45  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 39,   H: 44,   S: 44  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 44,   H: 41,   S: 43  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 39,   H: 38,   S: 52  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 44,   H: 35,   S: 51  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 39,   H: 42,   S: 50  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 43,   H: 39,   S: 49  -   F: many!!!		|  Last Action: work		| ...you lost some pennies....but Dudu helped, -1 points loss
| Life ->	M: 38,   H: 46,   S:

| Life ->	M: 41,   H: 37,   S: 51  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 36,   H: 44,   S: 50  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 41,   H: 41,   S: 49  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 36,   H: 48,   S: 48  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 39,   H: 45,   S: 47  -   F: many!!!		|  Last Action: work		| ...you lost some pennies....but Piz helped, -2 points loss
| Life ->	M: 44,   H: 42,   S: 46  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 39,   H: 49,   S: 45  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 44,   H: 46,   S: 44  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 39,   H: 53,   S: 43  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 44,   H: 50,   S: 42  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 49,   H: 44,   S: 41  -   F: many!!!		|  Last Action: work		| ...got hurt while cooking...but Leandr

let's find which is the problem printing some simulations. The result is not good but could be far worse.

In [33]:
ppo_agent_m.save_models(f"sd{seed}-{gamemode}-{difficulty}-pretrain")

### Step 4 - More Units (128)

In [34]:
num_steps = 1000000
# discount
gamma = 0.99
# lambda is the GAE parameter
lmbda = 0.95
# clipping parameter
#epsilon = 0.3
epsilon = 0.2
# entropy parameter
c2 = 0.005
batch_timesteps = 128
minibatch_size = 32
epochs = 4

lr_actor = 2.5e-4
lr_critic = 2.5e-4

gamemode = 'monopoly'
difficulty = 40

ppo_agent_m = agent.PPO_Agent(input_shape, 
                                n_outputs, 
                                gamma, 
                                lmbda, 
                                epsilon, 
                                c2, 
                                lr_actor, 
                                lr_critic, 
                                log=True, 
                                log_dir=f'{log_dir}-sd{seed}-s4-{gamemode}-{difficulty}-pro', 
                                env_max_timesteps=env_max_timesteps,
                                init_personal = True,
                                units=128)

vec_env_m.close()

vec_env_m = gymnasium.vector.make(
    'life_steps/LifeSteps-v0', num_envs=num_envs, asynchronous=True, max_timesteps = env_max_timesteps, difficulty = difficulty, gamemode = gamemode)

#ppo_agent_m.load_models("monopoly-15-pretrain")
m.reset()
gc.collect()

ppo_agent_m.play_n_timesteps(vec_env_m, m, num_steps, batch_timesteps, minibatch_size, epochs, difficulty=difficulty, gamemode = gamemode)

  0%|          | 0/7812 [00:00<?, ?it/s]

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -115.500

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -30.700

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -11.700

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -14.400

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -13.400

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: -40.400

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
>>> Evaluation <<<
	Mean Cumulative Rewards: 0.400

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

In [35]:
ppo_agent_m.save_models(f"sd{seed}-{gamemode}-{difficulty}-pro")

### Evaluation - 4

In [36]:
mcr, mean_actions = ppo_agent_m.evaluate(eval_episodes,  seeds=eval_seeds, render_mode='text', difficulty=difficulty, gamemode=gamemode)

print_evaluation(mcr, mean_actions)

STARTING EVALUATION

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Seed: 86344


| Life ->	M: 33,   H: 25,   S: 33  -   F: alone.., I just started playing!
| Life ->	M: 28,   H: 22,   S: 42  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 23,   H: 29,   S: 41  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 18,   H: 36,   S: 40  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 23,   H: 33,   S: 39  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 28,   H: 30,   S: 38  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 21,   H: 37,   S: 37  -   F: many!!!		|  Last Action: sport		| ...you lost some pennies....but Dudu helped, -2 points loss
| Life ->	M: 26,   H: 34,   S: 36  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 31,   H: 31,   S: 35  -   F: many!!!		|  Last Action: work		| all ok 
| L

| Life ->	M: 19,   H: 30,   S: 39  -   F: many!!!		|  Last Action: sport		| ...got hurt while cooking...but Piz helped, -4 points loss
| Life ->	M: 14,   H: 37,   S: 38  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 19,   H: 34,   S: 37  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 24,   H: 31,   S: 36  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 19,   H: 38,   S: 35  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 24,   H: 35,   S: 34  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 29,   H: 32,   S: 33  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 24,   H: 39,   S: 32  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 29,   H: 36,   S: 31  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 33,   S: 30  -   F: many!!!		|  Last Action: work		| ...you lost some pennies....but Dudu helped, -2 points loss
| Life ->	M: 27,   H: 40,   S: 29  -   F: many!!!		|  Last Acti

| Life ->	M: 22,   H: 24,   S: 39  -   F: alone..		|  Last Action: social		| all ok 
| Life ->	M: 17,   H: 21,   S: 48  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 12,   H: 28,   S: 47  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 17,   H: 25,   S: 46  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 12,   H: 32,   S: 45  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 17,   H: 29,   S: 44  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 20,   H: 26,   S: 43  -   F: many!!!		|  Last Action: work		| ...you lost some pennies....but Dave helped, -2 points loss
| Life ->	M: 15,   H: 33,   S: 42  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 20,   H: 30,   S: 41  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 25,   H: 27,   S: 40  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 20,   H: 34,   S: 39  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 25,   H: 31,  

| Life ->	M: 21,   H: 25,   S: 37  -   F: many!!!		|  Last Action: work		| ...you lost some pennies....but Leandro helped, -1 points loss
| Life ->	M: 16,   H: 32,   S: 36  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 21,   H: 29,   S: 35  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 26,   H: 25,   S: 34  -   F: many!!!		|  Last Action: work		| ...got hurt while cooking...but Dudu helped, -1 points loss
| Life ->	M: 21,   H: 32,   S: 33  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 26,   H: 29,   S: 32  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 21,   H: 32,   S: 31  -   F: many!!!		|  Last Action: sport		| ...got hurt while cooking...but Leandro helped, -4 points loss
| Life ->	M: 26,   H: 29,   S: 30  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 21,   H: 36,   S: 29  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 26,   H: 33,   S: 28  -   F: many!!!		|  Last Action: work		| all ok 
| Li

| Life ->	M: 13,   H: 27,   S: 43  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 18,   H: 22,   S: 42  -   F: many!!!		|  Last Action: work		| ...got hurt while cooking...but Leandro helped, -2 points loss
| Life ->	M: 13,   H: 29,   S: 41  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 18,   H: 26,   S: 40  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 23,   H: 23,   S: 39  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 18,   H: 30,   S: 38  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 23,   H: 27,   S: 37  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 18,   H: 34,   S: 36  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 23,   H: 31,   S: 35  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 28,   H: 28,   S: 34  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 23,   H: 32,   S: 33  -   F: many!!!		|  Last Action: sport		| ...got hurt while cooking...but Lea

...

| Life ->	M: 32,   H: 42,   S: 36  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 37,   H: 39,   S: 35  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 46,   S: 34  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 37,   H: 43,   S: 33  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 37,   S: 42  -   F: many!!!		|  Last Action: social		| ...got hurt while cooking...but Dave helped, -3 points loss
| Life ->	M: 27,   H: 44,   S: 41  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 32,   H: 41,   S: 40  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 37,   H: 38,   S: 39  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 41,   S: 38  -   F: many!!!		|  Last Action: sport		| ...got hurt while cooking...but Dave helped, -4 points loss
| Life ->	M: 37,   H: 38,   S: 37  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 45,   S: 36  -   F: many!!!		|  Last A

| Life ->	M: 36,   H: 51,   S: 47  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 41,   H: 48,   S: 46  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 46,   H: 45,   S: 45  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 41,   H: 52,   S: 44  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 46,   H: 49,   S: 43  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 41,   H: 56,   S: 42  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 46,   H: 53,   S: 41  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 41,   H: 60,   S: 40  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 42,   H: 57,   S: 39  -   F: many!!!		|  Last Action: work		| ...you lost some pennies....but Piz helped, -4 points loss
| Life ->	M: 47,   H: 54,   S: 38  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 42,   H: 51,   S: 47  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 47,   H: 48,   S:

| Life ->	M: 44,   H: 50,   S: 46  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 39,   H: 57,   S: 45  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 44,   H: 54,   S: 44  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 46,   H: 51,   S: 43  -   F: many!!!		|  Last Action: work		| ...you lost some pennies....but Piz helped, -3 points loss
| Life ->	M: 41,   H: 58,   S: 42  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 46,   H: 55,   S: 41  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 41,   H: 52,   S: 50  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 45,   H: 49,   S: 49  -   F: many!!!		|  Last Action: work		| ...you lost some pennies....but Dudu helped, -1 points loss
| Life ->	M: 40,   H: 56,   S: 48  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 45,   H: 53,   S: 47  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 50,   H: 50,   S: 46  -   F: many!!!		|  Last Act

| Life ->	M: 35,   H: 30,   S: 31  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 30,   H: 37,   S: 30  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 35,   H: 34,   S: 29  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 30,   H: 41,   S: 28  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 25,   H: 38,   S: 37  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 30,   H: 35,   S: 36  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 25,   H: 42,   S: 35  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 30,   H: 39,   S: 34  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 35,   H: 36,   S: 33  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 30,   H: 40,   S: 32  -   F: many!!!		|  Last Action: sport		| ...got hurt while cooking...but Dudu helped, -3 points loss
| Life ->	M: 35,   H: 37,   S: 31  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 30,   H: 44,   S

| Life ->	M: 32,   H: 31,   S: 34  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 27,   H: 38,   S: 33  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 32,   H: 35,   S: 32  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 27,   H: 42,   S: 31  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 32,   H: 39,   S: 30  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 37,   H: 36,   S: 29  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 32,   H: 33,   S: 38  -   F: many!!!		|  Last Action: social		| all ok 
| Life ->	M: 27,   H: 37,   S: 37  -   F: many!!!		|  Last Action: sport		| ...got hurt while cooking...but Dudu helped, -3 points loss
| Life ->	M: 32,   H: 34,   S: 36  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 27,   H: 41,   S: 35  -   F: many!!!		|  Last Action: sport		| all ok 
| Life ->	M: 32,   H: 38,   S: 34  -   F: many!!!		|  Last Action: work		| all ok 
| Life ->	M: 27,   H: 45,   S

# NOTEBOOK STOPS HERE